OCR WITH KERAS

In [72]:
import cv2
import os
import numpy as np
import imutils
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imutils import build_montages
from PIL import Image

from models import ResNet
import matplotlib
matplotlib.use("Agg")

EPOCHS = 50
INIT_LR = 1e-1
BS = 128

Datasets:

A-Z
http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

Load A-Z dataset:

In [73]:
def load_a_z_font_dataset(rootdir):
    data = []
    labels = []
    
    for (dirpath, dirnames, filenames) in os.walk(rootdir):
        for filename in filenames:
            
            labels.append(int(filename[8:11])-1)
            #print(int(filename[8:11])-11)
            
            image = cv2.imread(os.path.join(dirpath, filename))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            (thresh, bwImage) = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

            width, height = bwImage.shape
            totalpixels = width * height
            if cv2.countNonZero(bwImage) < totalpixels / 2:
                bwImage = np.invert(bwImage)

            (tH, tW) = bwImage.shape
            if tW > tH:
                bwImage = imutils.resize(bwImage, width=32)
            else:
                bwImage = imutils.resize(bwImage, height=32)

            (tH, tW) = bwImage.shape
            dX = int(max(0, 32 - tW) / 2.0)
            dY = int(max(0, 32 - tH) / 2.0)

            padded = cv2.copyMakeBorder(bwImage, top=dY, bottom=dY,
                left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
                value=(255, 255, 255))

            data.append(padded)                    

    return (data, labels)

Loading:

In [74]:
(data, labels) = load_a_z_font_dataset(r'C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\datasets\a_z_font')

data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")

data = np.expand_dims(data, axis=-1)
data /= 255.0

le = LabelBinarizer()

labels = le.fit_transform(labels)
ounts = labels.sum(axis=0)

Training assesment balance:

In [75]:
classTotals = labels.sum(axis=0)
classWeight = {}

for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

(trainX, testX, trainY, testY) = train_test_split(data,
                                                  labels, test_size=0.20, stratify=labels, random_state=42)

Preparing to train:

In [76]:
aug = ImageDataGenerator(rotation_range=10, zoom_range=0.05, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.15, horizontal_flip=False, fill_mode="nearest")

print("[INFO] compiling model...")

opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = ResNet.build(32, 32, 1, len(le.classes_), (3, 3, 3),
                     (64, 64, 128, 256), reg=0.0005)
                     
model.compile(loss="categorical_crossentropy",
              optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


C:\Users\alanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Training:

In [77]:
print("[INFO] training network...")

H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS), 
            validation_data=(testX, testY), 
            #steps_per_epoch=len(trainX),
            epochs=EPOCHS,
            class_weight=classWeight,
            verbose=1)

[INFO] training network...
Epoch 1/50


C:\Users\alanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


375/375 [==============================] - 745s 2s/step - loss: 3.9536 - accuracy: 0.3173 - val_loss: 1.9762 - val_accuracy: 0.5670
Epoch 2/50
375/375 [==============================] - 760s 2s/step - loss: 2.0484 - accuracy: 0.6646 - val_loss: 1.4575 - val_accuracy: 0.6530
Epoch 3/50
375/375 [==============================] - 752s 2s/step - loss: 1.5032 - accuracy: 0.7489 - val_loss: 1.0219 - val_accuracy: 0.7720
Epoch 4/50
375/375 [==============================] - 751s 2s/step - loss: 1.2889 - accuracy: 0.7778 - val_loss: 0.9374 - val_accuracy: 0.7853
Epoch 5/50
375/375 [==============================] - 754s 2s/step - loss: 1.1894 - accuracy: 0.7950 - val_loss: 0.8792 - val_accuracy: 0.8065
Epoch 6/50
375/375 [==============================] - 752s 2s/step - loss: 1.1297 - accuracy: 0.8034 - val_loss: 0.8699 - val_accuracy: 0.7993
Epoch 7/50
375/375 [==============================] - 754s 2s/step - loss: 1.0834 - accuracy: 0.8097 - val_loss: 0.8210 - val_accuracy: 0.8137
Epoch 8/50

Network evaluation:

In [78]:
labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
labelNames = [l for l in labelNames]

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=labelNames))


[INFO] evaluating network...
              precision    recall  f1-score   support

           A       0.93      0.97      0.95       315
           B       0.91      0.97      0.94       226
           C       0.77      0.52      0.62       246
           D       0.96      0.97      0.96       241
           E       0.93      0.95      0.94       292
           F       0.95      0.95      0.95       219
           G       0.96      0.95      0.95       232
           H       0.93      0.95      0.94       242
           I       0.86      0.67      0.75       264
           J       0.92      0.94      0.93       219
           K       0.90      0.96      0.93       222
           L       0.94      0.95      0.95       246
           M       0.92      0.95      0.93       233
           N       0.93      0.93      0.93       276
           O       0.73      0.46      0.57       280
           P       0.92      0.92      0.92       235
           Q       0.92      0.96      0.94       21

Saving the model and training history:

In [79]:
model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model"
model.save(model_path, save_format="h5")

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Trainning Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


Testing:

In [80]:
model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model"
model = load_model(model_path)

labelNames = "ACBDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
labelNames = [l for l in labelNames]

images = [] 
for i in np.random.choice(np.arange(0, len(testY)), size=(49,)):
    probs = model.predict(testX[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = labelNames[prediction[0]]
    #print(prediction)

    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)

    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)

    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)

    images.append(image)

montage = build_montages(images, (96, 96), (7, 7))[0]

cv2.imshow("OCR Results", montage)
cv2.waitKey(0)

C:\Users\alanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


-1

Predict:

In [81]:
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import numpy as np
import imutils
import cv2
import tensorflow as tf

tf.config.run_functions_eagerly(True)

model_path = r"C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model"

print("[INFO] loading OCR model...")
model = load_model(model_path)
print(model_path)

image_path = r"images\afastese.PNG"

image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

edged = cv2.Canny(blurred, 30, 150)

cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]

chars = []
            

for c in cnts:
	(x, y, w, h) = cv2.boundingRect(c)

	if (w >= 20 and w <= 150) and (h >= 15 and h <= 120):
		roi = gray[y:y + h, x:x + w]
		(t, bwImage) = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

		(tW, tH) = bwImage.shape
		totalpixels = tW * tH
		
		if cv2.countNonZero(bwImage) < totalpixels / 2:
			bwImage = np.invert(bwImage)

		if tW > tH:
			bwImage = imutils.resize(bwImage, width=32)
		else:
			bwImage = imutils.resize(bwImage, height=32)

		(tH, tW) = bwImage.shape
		dX = int(max(0, 32 - tW) / 2.0)
		dY = int(max(0, 32 - tH) / 2.0)

		padded = cv2.copyMakeBorder(t, top=dY, bottom=dY,
			left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
			value=(255, 255, 255))
		
		padded = cv2.resize(padded, (32, 32))
		padded = padded.astype("float32") / 255.0
		padded = np.expand_dims(padded, axis=-1)

		cv2.imshow("Image", padded)
		cv2.waitKey(0)

		chars.append((padded, (x, y, w, h)))

boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")


preds = model.predict(chars)

labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
labelNames = [l for l in labelNames]

for (pred, (x, y, w, h)) in zip(preds, boxes):
	i = np.argmax(pred)
	prob = pred[i]
	if prob > 0.8:
		print("prob:",prob)
		label = labelNames[i]

		print("[INFO] {} - {:.2f}%".format(label, prob * 100))
		cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
		cv2.putText(image, label, (x - 10, y - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

cv2.imshow("Image", image)
cv2.waitKey(0)

[INFO] loading OCR model...
C:\Projetos\Mestrado\Project II\SourceCode\TextIdentificationService\model\trained_ocr_full.model


-1